In [ ]:
# Imports et paths globaux
import rasterio
import numpy as np
import glob
import plotly.express as px
import plotly.graph_objs as go
import cv2
import pandas as pd
import geopandas as gpd
import os
import matplotlib.pyplot as plt


HOME = '/Users/u600141/OneDrive - La Francaise des Jeux/Data/constructions/'
PATHS_BDORTHO = HOME + 'images/BDORTHO_RGB/source/1_DONNEES_LIVRAISON_2022-06-00141/OHR_RVB_0M20_JP2-E080_LAMB93_D92-2021/*.jp2'
PATHS_BDTOPO = HOME + 'images/BDTOPO/source/1_DONNEES_LIVRAISON_2023-06-00117/BDT_3-3_SHP_LAMB93_D092-ED2023-06-15/*/*.shp'



In [ ]:
# Fonctions pour chargement base bdortho : Fichiers .tab et .jp2, tous sous le même répertoire

# Lecture d'un fichier .tab
# On cherche les lignes de type : (640000.00,6860000.00) (0,0) Label "Pt 1",
def load_tabfile(path):
    with open(path, 'r') as f:
        lines = f.readlines()

    coords = []
    for line in lines:
        if '(' in line and ')' in line:
            # récupère le premier tuple, eg. (640000.00,6860000.00)
            coord_str = line.split('(')[1].split(')')[0]
            coords.append(tuple(map(float, coord_str.split(','))))
    # coords contient les 4 tuples du fichier, eg. [(635000.0, 6860000.0), (640000.0, 6860000.0), (640000.0, 6855000.0), (635000.0, 6855000.0)]
    # rasterio.transform.from_bounds(west, south, east, north, width, height) retourne un objet de classe Affine 
    return rasterio.transform.from_bounds(*coords[0], *coords[2], len(coords[0]) // 2, len(coords) // 2) ## QUESTION rasterio.transform.from_bounds(*coords[0], *coords[2] 1 , 2

# Lecture d'un fichier .jp2
# retourne value (image), xrange et yrange(coordonnées des points extrêmes)
def load_image(path):
    with rasterio.open(path) as src:
        value = np.transpose(src.read(), [1,2,0]) # channel x hauteur x largeur --> hauteur x largeur x channel
        resolution = src.res
        bounds = src.bounds  
    transfrom = load_tabfile(path[:-3]+"tab") # TRANSFROM JAMAIS UTILISEE
    xrange = [bounds.left, bounds.right]
    yrange = [bounds.bottom, bounds.top]
    return value, xrange, yrange


In [ ]:
# Exemple de lecture d'une image parmi celles de la base ORTHO
paths_ortho = glob.glob(PATHS_BDORTHO)
value, xrange, yrange = load_image(paths_ortho[5])

print (f"Image exemple chargée : {paths_ortho[5].split('/')[-1]}")
print (f'value.shape = {value.shape}')
print (f'xrange = {xrange}, yrange = {yrange}')

In [ ]:
# Histogramme avant normalisation des couches de couleur
def display_histograms(value):
    colors = ['r', 'g', 'b']
    fig = plt.figure(figsize=(10, 4))
    for i in range(0, 3):
        fig.add_subplot(1, 3, i+1)
        plt.hist(value[..., i].flatten(), bins=255, color=colors[i])
    plt.show();

display_histograms(value)

In [ ]:
# Normalisation de chaque couche de couleur
# De plus, les valeurs parmi les 20% plus faibles ou plus élevées sont forcées à la valeur min ou max

def normalize_band(band, lower_percentile=20, upper_percentile=80):
    lower_value, upper_value = np.percentile(band, (lower_percentile, upper_percentile))
    print (f'lower value : {lower_value} - upper value : {upper_value}')
    return np.clip((band - lower_value) / (upper_value - lower_value), 0, 1)

value[...,0]=normalize_band(value[...,0])*255
value[...,1]=normalize_band(value[...,1])*255
value[...,2]=normalize_band(value[...,2])*255

# Affichage de l'histogramme après normalisation
display_histograms(value)

In [ ]:
# Chargement base BDTOPO : Fichiers .shp, avec un répertoire par thème (administratif, bâti, etc.)

# Lecture d'un fichier .shp
def load_shp(path):
    # Load data
    bdtopo = gpd.read_file(path)
    # Add Category and Type
    bdtopo.insert(0, 'Category', path.split('/')[-2])
    bdtopo.insert(1, 'Type', path.split('/')[-1][:-4])
    return bdtopo

# Lecture d'un fichier 
paths_shp = glob.glob(PATHS_BDTOPO)
bdtopo = load_shp(paths_shp[0])
for path in paths_shp[1:]:
    bdtopo = pd.concat([bdtopo, load_shp(path)])
bdtopo.reset_index(drop=True, inplace=True)

print (f'bdtopo chargé, taille {len(bdtopo)}')
bdtopo

QUELQUES ELEMENTS POUR QUALIFIER BDTOPO

In [ ]:
# Rapport exploratoire de BDORTHO : liste et taille des fichiers

for f in paths_ortho:
    print (f"{f.split('/')[-1]} - {round(os.stat(f).st_size / (1024*1024), 2)} Mo")

In [ ]:
# Rapport exploratoire de BDTOPO : recherche des dtypes et isna() et enregistrement dans un fichier .csv

cols = pd.DataFrame({'Type' : bdtopo.dtypes})
cols['isna()'] = round(bdtopo.isna().sum() / len(bdtopo), 2)
cols.to_csv('bdtopo_cols.csv')
cols

In [ ]:
# Contenu du champ geometry
print (bdtopo['geometry'])

In [ ]:
# Répartition du champ Category
print (bdtopo['Category'].value_counts(normalize=True))

In [ ]:
# Répartition du champ Type
print (bdtopo['Type'].value_counts(normalize=True))
print (len(bdtopo['Type'].value_counts()))

In [ ]:
# Nombre d'éléments par catégorie et type, avec exemple de géométrie qu'ils contiennent
bdtopo.groupby(['Category','Type']).agg(count=('Type','count'), geometry=("geometry", 'first'))

In [ ]:
# Exemple de contenu de chacune des colonnes renseignées

c = bdtopo.isna().sum()
cols = pd.DataFrame({'Column' : c.index, 'NAs' : c}).sort_values('NAs')
cols = cols.loc[cols['NAs'] < len(bdtopo)] # on supprime les colonnes non renseignées

for e in zip(cols['Column'], cols['NAs']):
    val = bdtopo.loc[~(bdtopo[e[0]].isna()), e[0]].iloc[0]
    print (e, val)


In [ ]:
# Ajout d'un champ GeoType indiquant le type d'objet géométrique
bdtopo['GeoType'] = bdtopo['geometry'].geom_type
print (bdtopo['GeoType'].value_counts(normalize=True))

In [ ]:
# Type de géométrie par catégorie
bdtopo.groupby(['Category', 'GeoType']).agg(count=('GeoType','count'))

In [ ]:
# Analyse du champ ID
print (bdtopo['ID'].value_counts().value_counts(normalize=True)) # Le champ est-il un vrai identifiant (unique) ?
print (bdtopo['ID'].value_counts().head()) # Détail des doublons
print (bdtopo[bdtopo['ID']=='PAIHABIT0000000331305224']) # Exemple d'un ID dupliqué

In [ ]:
# Autres champs catégoriels : value_counts()
print (bdtopo['ACQU_PLANI'].value_counts(normalize=True))
print ('\n', bdtopo['NATURE'].value_counts(normalize=True))
print ('\n', bdtopo['ETAT'].value_counts(normalize=True))
print ('\n', bdtopo['ACQU_ALTI'].value_counts(normalize=True))
print ('\n', bdtopo['ORIGIN_BAT'].value_counts(normalize=True))
print ('\n', bdtopo['LEGER'].value_counts(normalize=True))
print ('\n', bdtopo['USAGE1'].value_counts(normalize=True))


In [ ]:
# Exemple de figure de chaque type

# geometry : recherche d'un exemple par type de figure
shapes = bdtopo.groupby(['GeoType']).agg({'geometry' : 'first'}).reset_index()
print (shapes)

# Affichage des figures
n = round((len(shapes)**(1/2)+0.5))
fig = plt.figure(figsize=(8, 8))
for i in range(0, len(shapes)):
    s = fig.add_subplot(n, n, i+1)
    f = gpd.GeoSeries(shapes.iloc[i,1]) # iloc retourne une shape et pas une GeoSeries, au contraire de loc
    f.plot(ax=s)
    s.set_title(str(shapes.iloc[i,1]).split('(')[0])
    plt.xticks([])
    plt.yticks([])
plt.subplots_adjust(wspace=0.4, hspace=0.4)

plt.show();

DOCUMENTATION et LIENS UTILES

https://geoservices.ign.fr/sites/default/files/2021-07/DC_BDTOPO_3-0.pdf
https://shapely.readthedocs.io/en/stable/reference/shapely.Polygon.html


FONCTIONS UTILES

In [ ]:
# Fonctions utiles pour le positionnement : isInMap(), convert_centroid(), convert_polygon()

map_size = [5000,5000]

# Variables globales utilisées :
#    map_size : taille finale de l'image à afficher
#    xrange et yrange : limites de l'image initiale (chargée précédemment)

# isInMap : retourne True si le centroïde du polygone est dans l'image de limites (xrange, yrange)
def isInMap(xrange, yrange):
    def my_function(polynom):
        x, y = polynom.centroid.x, polynom.centroid.y
        if xrange[0]<x and xrange[1]>x and yrange[0]<y and yrange[1]>y:
            return True
        else :
            return False
        
    return my_function

# convert_centroid : retourne les coordonnées du centroïde converties dans la taille de l'image à afficher (de taille map_size)
def convert_centroid(map_size, xrange, yrange):
    def my_function(polygon):
        x, y = polygon.centroid.x, polygon.centroid.y
        x_new = (x - xrange[0])/(xrange[1]-xrange[0])*map_size[0]
        y_new = map_size[1] - (y - yrange[0])/(yrange[1]-yrange[0])*map_size[1]
        return [x_new, y_new]
    
    return my_function

# convert_polygon : retourne les listes de coordonnées de la shape converties dans la taille de l'image à afficher (de taille map_size)
def convert_polygon(map_size, xrange, yrange):
    def my_function(polygon):
        if polygon.wkt.lower()[:7]=='polygon':
            x, y = polygon.exterior.coords.xy
            x = x.tolist()
            y = y.tolist()
        elif polygon.wkt[:10]=='LINESTRING':
            x, y = polygon.coords.xy
            x = x.tolist()
            x += x[::-1]
            y = y.tolist()
            y += y[::-1]       
        else :
            x = [1,2]
            y = [1,2]
        x = np.array(x)
        y = np.array(y) 
        x_new = (x - xrange[0])/(xrange[1]-xrange[0])*map_size[0]
        y_new = map_size[1] - (y - yrange[0])/(yrange[1]-yrange[0])*map_size[1]
        
        return [x_new, y_new]
    
    return my_function

# On crée bdtopo_zone : ensemble des shapes de la base qui ont leur centroïde dans l'image chargée
bdtopo_zone = bdtopo[bdtopo['geometry'].apply(isInMap(xrange, yrange))].copy()
print('Avant :', bdtopo.shape, 'Après :', bdtopo_zone.shape)

# Ajout du centroïde
bdtopo_zone['centroid'] = bdtopo_zone['geometry'].apply(convert_centroid(map_size, xrange, yrange))
bdtopo_zone['xcentroid'] = bdtopo_zone['centroid'].apply(lambda x : x[0])
bdtopo_zone['ycentroid'] = bdtopo_zone['centroid'].apply(lambda x : x[1])

# On distingue la zone en bdtopo_point (ensemble des points) et bdtopo_zone (autres shapes)
bdtopo_point = bdtopo_zone[bdtopo['geometry'].apply(lambda x : x.wkt.lower()[:5]=="point")]
bdtopo_zone = bdtopo_zone[bdtopo_zone['geometry'].apply(lambda x : x.wkt.lower()[:7]=="polygon" or x.wkt[:10]=="LINESTRING")]

# Ajout des colonnes 'polygone' avec les polygones dans la taille de l'image à afficher
bdtopo_zone['polygon'] = bdtopo_zone['geometry'].apply(convert_polygon(map_size, xrange, yrange))
bdtopo_zone['xpolygon'] = bdtopo_zone['polygon'].apply(lambda x : x[0])
bdtopo_zone['ypolygon'] = bdtopo_zone['polygon'].apply(lambda x : x[1])

bdtopo_zone.head()

In [ ]:
# Fonctions utiles pour le positionnement (suite) : generate_xy_polygons(), generate_x_polygons()

# generate_xy_polygons :  transformation des coordonnées d'une zone en listes + inversion de la coordonnée y
def generate_xy_polygons(bdtopo_area):

    list_x = []
    for xpoly in bdtopo_area['xpolygon']:
        list_x.extend(xpoly.tolist() + [None])
    list_x = list_x[:-1]
    
    list_y = []
    for ypoly in bdtopo_area['ypolygon']:
        ypoly = map_size[1]-ypoly
        list_y.extend(ypoly.tolist() + [None])
    list_y = list_y[:-1]
    
    return list_x, list_y

# generate_x_polygons : transformation du Array en liste
def generate_x_polygons(xdata):
    list_x = []
    for xpoly in xdata:
        list_x.extend(xpoly.tolist() + [None])
    list_x = list_x[:-1]
    return list_x

# Création de bdtopo_zone_agregate, qui comporte une ligne par type, avec toutes les figures de ce type dans les mêmes champs x et y
bdtopo_zone_agregate = bdtopo_zone.groupby('Type').agg({'xpolygon':list, 'ypolygon':list})
bdtopo_zone_agregate['xpolygon_ready'] = bdtopo_zone_agregate['xpolygon'].apply(generate_x_polygons)
bdtopo_zone_agregate['ypolygon_ready'] = bdtopo_zone_agregate['ypolygon'].apply(generate_x_polygons)
bdtopo_zone_agregate

In [ ]:
# Création de bdtopo_point_agregate, qui comporte une ligne par type, avec tous les points de ce type

bdtopo_point_agregate = bdtopo_point.groupby('Type').agg({'xcentroid':list, 'ycentroid':list})
bdtopo_point_agregate

In [ ]:
# show_map() : affiche la carte avec les shapes

def show_map(value, scatters_data, scatters_list_name, points_data, points_list_name,
             showOrtho=True, showTopo=True):

    if (showOrtho):
        image = px.imshow(cv2.resize(value, (5000,5000)))
    
    if (showTopo):
        points = []
        
        for i, (list_x, list_y) in enumerate(scatters_data):
            # Ajouter des points
            point = go.Scatter(
                x=list_x,
                y=list_y,
                fill="toself",
                name=scatters_list_name[i],
            #     fillcolor="blue"
    
            )
            points.append(point)
            
        for i, (list_x, list_y) in enumerate(points_data):
            # Ajouter des points
            point = go.Scatter(
                x=list_x,
                y=list_y,
                mode='markers',
                marker=dict( size=5),
                name=points_list_name[i]
            )
            points.append(point)

    # Créer la figure
    if (showOrtho & showTopo):
        fig = go.Figure(data=[image.data[0]] + points)
    elif (showOrtho):
        fig = go.Figure(data=[image.data[0]])
    else:
        fig = go.Figure(data=points)
        
    fig.update_xaxes(range=[0,5000])
    fig.update_yaxes(range=[0,5000])
    
    fig.update_layout(
        autosize=False,
        width=1000,
        height=800,)

    # Afficher la figure
    fig.show()

show_map(value,
         bdtopo_zone_agregate[['xpolygon_ready', 'ypolygon_ready']].values,
         bdtopo_zone_agregate.index,
         bdtopo_point_agregate[['xcentroid', 'ycentroid']].values,
         bdtopo_point_agregate.index)

In [ ]:
# Affichage de l'image sans la topographie

show_map(value, None, None, None, None, showTopo=False)

In [ ]:
# Affichage de la topographie sans l'image
show_map(value,
         bdtopo_zone_agregate[['xpolygon_ready', 'ypolygon_ready']].values,
         bdtopo_zone_agregate.index,
         bdtopo_point_agregate[['xcentroid', 'ycentroid']].values,
         bdtopo_point_agregate.index,
        showOrtho=False)